# Graphs


## Enviroment Set-up

In [1]:
import sys
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

from dash_layout import layout
from dash import dcc, html
import data_handling  
import plot_functions
import pandas as pd
import numpy as np
import warnings
import json
import re
import io

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
fig = go.Figure(layout=dict(template='plotly')) ## solves invalid value error from plotly
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [3]:
app.layout = layout  ## use the layout setting in dash_layout.py

## Dashboard callbacks

In [4]:
@app.callback(Output('output-json-data-upload', 'data'), 
              Output('timestamp_num', 'data'),
              Output('upload-data-json', 'style'),
              Input('upload-data-json','style'),
              Input('upload-data-json', 'contents'),
              State('upload-data-json', 'filename'),
              State('upload-data-json', 'last_modified'))
def update_json_output(style,list_of_contents, list_of_names, list_of_dates):
    
    if list_of_contents is not None:
        timestamp_num= 0
        data = data_handling.parse_contents(list_of_contents, list_of_names, list_of_dates)
        for bondtype in data:
            for bond in data[bondtype]:
                temp= max([ int(x) for x in data[bondtype][bond]['ps']])
                if timestamp_num < temp:
                    timestamp_num = temp
        style['background-color']='green'
        return data,timestamp_num//10,style
    else: 
        fake = '{}'
        return json.loads(fake),0,style

In [5]:
@app.callback(Output('output-csv-data-upload', 'data'),
              Output('upload-data-csv', 'style'),
              Input('upload-data-csv','style'),
              Input('upload-data-csv', 'contents'),
              State('upload-data-csv', 'filename'),
              State('upload-data-csv', 'last_modified'))
def update_csv_output(style, list_of_contents, list_of_names, list_of_dates):
    
    if list_of_contents is not None:
        style['background-color']='green'
        return data_handling.parse_contents(list_of_contents, list_of_names, list_of_dates),style
    else: 
        fake = '{}'
        return json.loads(fake),style

In [6]:
@app.callback(Output('output-csv-rcpt-upload', 'data'),
              Output('upload-rcpt-csv', 'style'),
              Input('upload-rcpt-csv','style'),
              Input('upload-rcpt-csv', 'contents'),
              State('upload-rcpt-csv', 'filename'),
              State('upload-rcpt-csv', 'last_modified'))
def update_csv_output(style,list_of_contents, list_of_names, list_of_dates):
    
    if list_of_contents is not None:
        style['background-color']='green'
        return data_handling.parse_contents(list_of_contents, list_of_names, list_of_dates),style
    else: 
        fake = '{}'
        return json.loads(fake),style

In [7]:
@app.callback(Output('atom', 'options'), Output('mapatoms','data'), Input('output-csv-data-upload', 'data'))
def update_lig_dropdown_output(lig_atoms):
    if not lig_atoms : return [{"label": '', "value": ''}], []
    lig_atoms =  pd.read_json(lig_atoms, orient='split')
    
    atoms = lig_atoms["ATOM"].to_list()
    mapatoms = dict(zip(lig_atoms["ATOM"], lig_atoms["ATOM_ID"]))
    atomops = [{"label": atom, "value": atom}  for atom in atoms]
    return atomops, mapatoms
    

In [8]:
@app.callback(Output('recs', 'options'), Input('output-csv-rcpt-upload', 'data'))
def update_prot_dropdown_output(res_atoms):
    if not res_atoms: return [{"label": '', "value": ''}]
    res_atoms =  pd.read_json(res_atoms, orient='split').drop_duplicates(subset =["RES_TYPE","RES_NUM"],keep='first')
    
    recs = list(map(lambda x: x[0] + str(x[1]) , zip(res_atoms['RES_TYPE'],  res_atoms['RES_NUM'])))
    recsops = [ {"label": rec, "value": rec} for rec in recs]
    return recsops

## Results

In [9]:
@app.callback(Output("bondp", "figure"), Output("atoms_permanences_per_bondtype","data"), [Input('ts_slider', "value"), Input('output-json-data-upload', "data"), Input('output-csv-data-upload', "data"),Input('timestamp_num', "data") ])
def make_bondp(ts, json_data, lig_atoms, timestamp_num):
   
    if not lig_atoms  or not json_data : 
        atoms_permanences_per_bondtype = pd.DataFrame()
    else:
        lig_atoms =  pd.read_json(lig_atoms, orient='split')
        atoms_permanences_per_bondtype = data_handling.get_atoms_permanence(np.asarray(lig_atoms.iloc[:,0:2].astype('str')),json_data , int(timestamp_num))
    return plot_functions.atoms_permanence_to_barplot(atoms_permanences_per_bondtype, ts), atoms_permanences_per_bondtype.to_json(date_format='iso', orient='split')

In [10]:
@app.callback(Output("prembond", "figure"), Output("ligands_intercations_per_bondtype","data"), [Input('output-json-data-upload', "data"), Input('output-csv-data-upload', "data")])
def make_prembond(json_data, lig_atoms):

    if not lig_atoms or not json_data : 
        ligands_intercations_per_bondtype = pd.DataFrame()
    else:
        lig_atoms =  pd.read_json(lig_atoms, orient='split')
        ligands_intercations_per_bondtype = data_handling.getBondPermanence(json_data , np.asarray(lig_atoms.iloc[:,0:2].astype('str')))

        
    return plot_functions.permanence_to_bar(ligands_intercations_per_bondtype), ligands_intercations_per_bondtype.to_json(date_format='iso', orient='split')

In [11]:

@app.callback([Output("nucleomap", "figure"), Output("aminomap", "figure"), Output("nuc_csv","data"), Output("ammin_csv","data")], [Input('bondtype', "value"), Input('output-json-data-upload', "data"),Input('output-csv-data-upload', "data"),Input('timestamp_num', "data")])
def makeboth(btype,json_data,lig_atoms,timestamp_num):
    if lig_atoms and json_data:
        lig_atoms =  pd.read_json(lig_atoms, orient='split')
        nucleotids, ammin = data_handling.atom_receptor2(json_data, np.asarray(lig_atoms.iloc[:,0:2].astype('str')), btype, int(timestamp_num))
    else:
        nucleotids = pd.DataFrame()
        ammin = pd.DataFrame()
    
    ntitle = '('+', '.join(btype)+')' if len(btype) < 4 else 'All'
    nucl = plot_functions.pltres_lig_heatmap(nucleotids,"DNA "+ntitle,int(timestamp_num), min_val=1)
    am = plot_functions.pltres_lig_heatmap(ammin,"aminoacid "+ntitle,int(timestamp_num), min_val=1)
    return nucl, am , nucleotids.to_json(date_format='iso', orient='split'), ammin.to_json(date_format='iso', orient='split')


In [12]:
@app.callback(Output("dna-lig-activity", "data") , Output("prot-lig-activity", "data"), [Input('inter_slider', "value"), Input('output-json-data-upload', "data"), Input('output-csv-data-upload', "data"),Input('timestamp_num', "data")])
def make_resactivities(ts, json_data,lig_atoms,timestamp_num):
    if not lig_atoms or not json_data: return pd.DataFrame().to_json(date_format='iso', orient='split'),pd.DataFrame().to_json(date_format='iso', orient='split')
    lig_atoms =  pd.read_json(lig_atoms, orient='split')
    receptors_interaction_timeline = data_handling.atom_receptor_perma(json_data, np.asarray(lig_atoms.iloc[:,0:2].astype('str')), int(timestamp_num))


    dna_residues = []
    aminoacid_residues = []
    for res in receptors_interaction_timeline['Receptor']:
        if not re.match('[A-Z]{3}', res): dna_residues.append(res)
        else: aminoacid_residues.append(res)
        
    dna_residues_interaction_timeline = receptors_interaction_timeline[receptors_interaction_timeline.Receptor.isin(dna_residues)]
    aminoacid_residues_interaction_timeline = receptors_interaction_timeline[receptors_interaction_timeline.Receptor.isin(aminoacid_residues)]
    
    return dna_residues_interaction_timeline.to_json(date_format='iso', orient='split'), aminoacid_residues_interaction_timeline.to_json(date_format='iso', orient='split')
    
    

In [13]:
@app.callback(Output("resactivity", "figure"), [Input('inter_slider', "value"), Input("dna-lig-activity", "data"),Input('timestamp_num', "data")])
def make_resactivity(ts, json_data, timestamp_num):
    if json_data :
        data =  pd.read_json(json_data, orient='split')
    else: data = pd.DataFrame()
    return plot_functions.plt_res_activity(data, int(timestamp_num), threshold=ts)

In [14]:
@app.callback(Output("resactivityamino", "figure"), [Input('inter_slider', "value"), Input("prot-lig-activity", "data"),Input('timestamp_num', "data")])
def make_resactivityamino(ts, json_data,timestamp_num):
    if json_data:
        data =  pd.read_json(json_data, orient='split')
    else: data = pd.DataFrame()
    return plot_functions.plt_res_activity(data, int(timestamp_num), threshold=ts)

In [15]:
@app.callback(Output("atompermbondtype", "figure"), Output("atom_data","data"), [Input('atom', "value"), Input('output-json-data-upload', "data"),Input('mapatoms', "data"),Input('timestamp_num', "data")])
def make_atompermbondtype(atom,json_data,mapatoms,timestamp_num):
    
    if json_data and atom!=None :
        bond_permanence = data_handling.permanence_bond(json_data, str(mapatoms[atom]))
    else: bond_permanence = pd.DataFrame()
       
    return plot_functions.plt_res_activity(bond_permanence, int(timestamp_num), threshold=0.1), bond_permanence.to_json(date_format='iso', orient='split')

In [16]:
@app.callback(Output("atompermbondtype2", "figure"), Output("atom_res_data","data"), [Input('atom', "value"), Input('recs', "value"),Input('output-json-data-upload', "data"),Input('mapatoms', "data"),Input('timestamp_num', "data")])
def make_atompermbondtype2(atom, rec, json_data,mapatoms,timestamp_num):
    if json_data!=None and atom!= None and rec!= None:
        atom_rec = data_handling.onelig_oneres(json_data,str(mapatoms[atom]), rec, int(timestamp_num))
    else: atom_rec = pd.DataFrame()
    if atom == None or rec== None : 
        atom = ''
        rec = ''
    return plot_functions.plt_res_activity(atom_rec,int(timestamp_num), what=atom+"-"+rec, threshold=0),atom_rec.to_json(date_format='iso', orient='split')

In [17]:
@app.callback(
    Output("download-bondp-csv", "data"),
    Input("bondp_csv", "n_clicks"), State("atoms_permanences_per_bondtype","data"),
    prevent_initial_call=True,
)

def func(n_clicks,atoms_permanences_per_bondtype):
    return dcc.send_data_frame(pd.read_json(atoms_permanences_per_bondtype, orient='split').to_csv, "atoms_permanences_per_bondtype.csv")


In [18]:
@app.callback(
    Output("download-prembond-csv", "data"),
    [Input("prembond_csv", "n_clicks")], State("ligands_intercations_per_bondtype","data"),
    prevent_initial_call=True,
)

def func(n_clicks,ligands_intercations_per_bondtype):
    return dcc.send_data_frame(pd.read_json(ligands_intercations_per_bondtype, orient='split').to_csv, "ligands_intercations_per_bondtype.csv")

In [19]:
@app.callback(
    Output("download-nucleomap-csv", "data"),
    [Input("nucleomap_csv", "n_clicks")], State("nuc_csv","data"),
    prevent_initial_call=True,
)

def func(n_clicks,nuc_csv):
    return dcc.send_data_frame(pd.read_json(nuc_csv, orient='split').to_csv, "nucleomap_all.csv")

In [20]:
@app.callback(
    Output("download-aminomap-csv", "data"),
    [Input("aminomap_csv", "n_clicks")], State("ammin_csv","data"),
    prevent_initial_call=True,
)

def func(n_clicks, ammin):
    return dcc.send_data_frame(pd.read_json(ammin, orient='split').to_csv, "aminomap_all.csv")

In [21]:
@app.callback(
    Output("download-resactivity-csv", "data"),
    Input("resactivity_csv", "n_clicks"), State("dna-lig-activity","data"),
    prevent_initial_call=True,
)

def func(n_clicks,dnaligactivity):
    return dcc.send_data_frame(pd.read_json(dnaligactivity, orient='split').to_csv , "dna_residues_interaction_timeline.csv")

In [22]:
@app.callback(
    Output("download-resactivityamino-csv", "data"),
    Input("resactivityamino_csv", "n_clicks"), State("prot-lig-activity","data"),
    prevent_initial_call=True,
)

def func(n_clicks,protligactivity):
    return dcc.send_data_frame(pd.read_json(protligactivity, orient='split').to_csv , "aminoacid_residues_interaction_timeline.csv")

In [23]:
@app.callback(
    Output("download-atom_activity-csv", "data"),
    Input("atom_csv", "n_clicks"), State("atom_data","data"),
    prevent_initial_call=True,
)

def func( n_clicks, atom_data):
    return dcc.send_data_frame(pd.read_json(atom_data, orient='split').to_csv, "atom_data.csv")

In [24]:
@app.callback(
    Output("download-atom_res_activity-csv", "data"),
    Input("atom_res_csv", "n_clicks"), State("atom_res_data","data"),
    prevent_initial_call=True,
)

def func( n_clicks, atom_res_data):
    return dcc.send_data_frame(pd.read_json(atom_res_data, orient='split').to_csv, "atom_res_data.csv")

In [25]:
app.run_server(mode='external',port=8085)

Dash app running on http://127.0.0.1:8085/
